In [2]:
from forecasting_functions import *

# importing the data
Stijn = True
fullYearpath = None
av_temperaturepath = None

if Stijn:
    fullYearpath = "D:\Onedrive\Leuven\Final project\data\Forecasting_writtendata\FullYear.csv"
    av_temperaturepath = "D:\Onedrive\Leuven\Final project\data\weather-avg.csv"
else:
    raise Exception("Put some paths here.")
fullYeardata = pd.read_csv(fullYearpath,index_col= "date",parse_dates= True)
av_temperature = pd.read_csv(av_temperaturepath,index_col='meter_id')
av_temperature = av_temperature.transpose()
av_temperature.index = pd.to_datetime(av_temperature.index)



name = fullYeardata.columns[1]
TS = fullYeardata[name]
temperature = av_temperature[name]



# all the training dates are present, dates from the test set can be missing.

# normalize the data --> min/max method (when using a single time serie)
temperature_norm,scaler_temperature = norm_forcast(temperature)
temperature_norm = substitute_missing_values_temperature(temperature_norm)
TS_norm,scaler_history = norm_forcast(TS)
temp = TS_norm[TS_norm.index.month == 1]
# temp = TS_norm[TS_norm.index.month != 12]
# test = TS_norm[TS_norm.index.month == 12]

# remove from the test set all the days that contain nan values -> only estimate real days
# test.dropna(inplace=True)

# substitute the missing values
temp = substitute_missing_values(temp)
training = temp[0:336]
validation = temp[336:384]
test = temp[384:528]
# training = temp[temp.index.month != 11]
# validation = temp[temp.index.month == 11]
TS_norm_full = substitute_missing_values(TS_norm.copy(deep=True))

amount of missing values: 0. 

amount of missing values: 0. 

amount of missing values: 0. 



In [3]:
X,y = input_output_LSTM(training, temperature_norm, 3)
X_val, y_val = input_output_LSTM(validation, temperature_norm, 3)
X.shape


(333, 3, 59)

In [4]:
from keras.layers import Dense,  LSTM,Embedding
from keras.models import Sequential, save_model, load_model, Input, Model
from keras import regularizers
from keras.callbacks import EarlyStopping,ModelCheckpoint, History
inputs1 = Input(shape=(X.shape[1], X.shape[2]))
lstm1, state_h, state_c = LSTM(1, return_state=True, return_sequences=True)(inputs1)

model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
print(model.predict(np.zeros((1,3,59))))

Using TensorFlow backend.



[array([[[0.],
        [0.],
        [0.]]], dtype=float32), array([[0.]], dtype=float32), array([[0.]], dtype=float32)]


In [18]:

from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(batch_shape=(1,3, 1))
lstm1, state_h, state_c = LSTM(1, stateful= True, return_state= True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
# make and show prediction
for row in data:
    row = row.reshape(1,3,1)
    print(model.predict(row))
    # model.reset_states()
    print(10*'-')

[array([[0.]], dtype=float32), array([[0.]], dtype=float32), array([[0.]], dtype=float32)]
----------
[array([[-0.01457311]], dtype=float32), array([[-0.01457311]], dtype=float32), array([[-0.03429388]], dtype=float32)]
----------
[array([[-0.00350447]], dtype=float32), array([[-0.00350447]], dtype=float32), array([[-0.00699666]], dtype=float32)]
----------
[array([[-0.00068961]], dtype=float32), array([[-0.00068961]], dtype=float32), array([[-0.00137875]], dtype=float32)]
----------
[array([[-0.02256516]], dtype=float32), array([[-0.02256516]], dtype=float32), array([[-0.06389377]], dtype=float32)]
----------


In [17]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_state= True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
data = data.reshape((5,3,1))
# make and show prediction
# for row in data:
#     row = row.reshape(1,3,1)
output = model.predict(data)
print(model.predict(data))
print(10*'-')


[array([[0.        ],
       [0.03194633],
       [0.        ],
       [0.        ],
       [0.07881062]], dtype=float32), array([[0.        ],
       [0.03194633],
       [0.        ],
       [0.        ],
       [0.07881062]], dtype=float32), array([[0.        ],
       [0.05887042],
       [0.        ],
       [0.        ],
       [0.13532467]], dtype=float32)]
----------


In [13]:

from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array

# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))



[array([[-0.00906921]], dtype=float32), array([[-0.00906921]], dtype=float32), array([[-0.02078841]], dtype=float32)]
